In [1]:
import tensorflow as tf
from transformers import TFAutoModel, AutoTokenizer
from datasets import load_dataset

In [2]:
model = TFAutoModel.from_pretrained("bert-base-uncased")

2025-05-27 15:22:46.826719: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/programs/cuda/11.8/lib64:/opt/programs/openMPI/4.1.6/lib:/opt/programs/hwloc/2.12.0/lib:/u/dssc/ppette00/miniconda3/lib::
2025-05-27 15:22:46.826751: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.Layer

In [3]:
model

In [4]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [9]:
inputs = tokenizer(['Hello world', 'Hi how are you'], padding=True, truncation=True, return_tensors='tf')

In [8]:
inputs

{'input_ids': <tf.Tensor: shape=(2, 6), dtype=int32, numpy=
array([[ 101, 7592, 2088,  102,    0,    0],
       [ 101, 7632, 2129, 2024, 2017,  102]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(2, 6), dtype=int32, numpy=
array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(2, 6), dtype=int32, numpy=
array([[1, 1, 1, 1, 0, 0],
       [1, 1, 1, 1, 1, 1]], dtype=int32)>}

In [10]:
output_model = model(inputs)

In [11]:
output_model

TFBaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=<tf.Tensor: shape=(2, 6, 768), dtype=float32, numpy=
array([[[-0.16888423,  0.13606375, -0.1394004 , ..., -0.62511253,
          0.0521724 ,  0.3671451 ],
        [-0.3632746 ,  0.14122003,  0.8799881 , ...,  0.10433009,
          0.28875768,  0.3726794 ],
        [-0.6985945 , -0.6987971 ,  0.06450248, ..., -0.2210367 ,
          0.0098687 , -0.59397894],
        [ 0.8309827 ,  0.12366682, -0.15119113, ...,  0.10309644,
         -0.6779267 , -0.26285204],
        [-0.40266672, -0.01928207,  0.5732504 , ..., -0.2065682 ,
          0.02338546,  0.2012631 ],
        [-0.6228408 , -0.27453476,  0.18117644, ..., -0.12944858,
         -0.038391  , -0.05733228]],

       [[ 0.0928655 , -0.02636368, -0.12239344, ..., -0.2106351 ,
          0.1738637 ,  0.17250948],
        [ 0.4074199 , -0.05930912,  0.55234754, ..., -0.67905647,
          0.65557426, -0.29456505],
        [-0.21155237, -0.6858646 , -0.46280938, ...,  0.15278468

In [12]:
dataset = load_dataset("stanfordnlp/sst2")

README.md:   0%|          | 0.00/5.27k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/3.11M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/72.8k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/148k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

In [13]:
dataset

DatasetDict({
    train: Dataset({
        features: ['idx', 'sentence', 'label'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['idx', 'sentence', 'label'],
        num_rows: 872
    })
    test: Dataset({
        features: ['idx', 'sentence', 'label'],
        num_rows: 1821
    })
})

In [14]:
def tokenize(batch):
    return tokenizer(batch['sentence'], padding=True, truncation=True)

In [15]:
sentences_encoded = dataset.map(tokenize, batched=True, batch_size=None)

Map:   0%|          | 0/67349 [00:00<?, ? examples/s]

Map:   0%|          | 0/872 [00:00<?, ? examples/s]

Map:   0%|          | 0/1821 [00:00<?, ? examples/s]

In [16]:
sentences_encoded

DatasetDict({
    train: Dataset({
        features: ['idx', 'sentence', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['idx', 'sentence', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 872
    })
    test: Dataset({
        features: ['idx', 'sentence', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1821
    })
})

In [17]:
# setting 'input_ids', 'attention_mask', 'token_type_ids', and 'label'
# to the tensorflow format. Now if you access this dataset you will get these
# columns in `tf.Tensor` format

sentences_encoded.set_format(
    'tf', 
    columns=['input_ids', 'attention_mask', 'token_type_ids', 'label']
)

# setting BATCH_SIZE to 64.
BATCH_SIZE = 64

def order(inp):
    '''
    This function will group all the inputs of BERT
    into a single dictionary and then output it with
    labels.
    '''
    data = list(inp.values())
    return {
        'input_ids': data[1],
        'attention_mask': data[2],
        'token_type_ids': data[3]
    }, data[0]

# converting train split of `emotions_encoded` to tensorflow format
train_dataset = tf.data.Dataset.from_tensor_slices(sentences_encoded['train'][:])
# set batch_size and shuffle
train_dataset = train_dataset.batch(BATCH_SIZE).shuffle(1000)
# map the `order` function
train_dataset = train_dataset.map(order, num_parallel_calls=tf.data.AUTOTUNE)

# ... doing the same for test set ...
test_dataset = tf.data.Dataset.from_tensor_slices(sentences_encoded['test'][:])
test_dataset = test_dataset.batch(BATCH_SIZE)
test_dataset = test_dataset.map(order, num_parallel_calls=tf.data.AUTOTUNE)

In [21]:
inp, out = next(iter(train_dataset)) # a batch from train_dataset
print(inp, '\n\n', out)

{'input_ids': <tf.Tensor: shape=(64, 66), dtype=int64, numpy=
array([[  101,  2019,  2125, ...,     0,     0,     0],
       [  101,  8183,  3367, ...,     0,     0,     0],
       [  101,  2097,  2022, ...,     0,     0,     0],
       ...,
       [  101, 20996, 14227, ...,     0,     0,     0],
       [  101,  1037,  2143, ...,     0,     0,     0],
       [  101,  1010,  1036, ...,     0,     0,     0]])>, 'attention_mask': <tf.Tensor: shape=(64, 66), dtype=int64, numpy=
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])>, 'token_type_ids': <tf.Tensor: shape=(64, 66), dtype=int64, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]])>} 

 tf.Tensor(
[1 0 1 1 0 1 0 1 1 1 0 1 1 

In [22]:
class BERTForClassification(tf.keras.Model):
    def __init__(self, bert_model, num_classes):
        super().__init__()
        self.bert = bert_model
        self.fc = tf.keras.layers.Dense(num_classes, activation='softmax')
    def call(self, inputs):
        x = self.bert(inputs)[1]
        return self.fc(x)

In [24]:
classifier = BERTForClassification(model, num_classes=2)

classifier.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

In [ ]:
history = classifier.fit(
    train_dataset,
    epochs=3,
)

Epoch 1/3
   6/1053 [..............................] - ETA: 6:27:24 - loss: 0.7010 - accuracy: 0.5312

In [ ]:
classifier.evaluate(test_dataset)